# Getting available actions

**Note** this notebook requres that there are pending actions in the database. So go create some first. 

Actions are configured in `app\configurations` directory. When the UI makes a request to get the actions available for an object, the `app/ajaxviews/actions.py` loads the yaml, and interprets what actions that the selected object is available to take. Those actions are sent to the user. 

### Two times when actions come up:
but only one discussed here. 
* ~~web interaction that creates jobs~~
* **azure functions that resolve actions**

In [1]:
import sys
import numpy as np
import pandas as pd


sys.path.append("../..")
import yaml, ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()


from app.connectors import cmdb_graph
from app.objects import time as t


c = cmdb_graph.CosmosdbClient()


executing local windows deployment
something wrong with your query: <class 'Exception'>


In [2]:
time = t.Time(c)
time.get_current_UTU()
time

< time at UTU:69658 >

In [4]:
time.get_global_actions()
actions_df = pd.DataFrame(time.actions)
actions_df

,action,job,agent
0,"{'type': 'fabricating', 'comment': 'Yetam Fen:...","{'status': 'pending', 'userguid': '8d5b667f-b2...","{'isIdle': 'false', 'name': 'Yetam Fen', 'obji..."


All actions require both an agent, job and action. The properties of the action determine how it affects other things. 


In [5]:
for i in actions_df.index:
    action = t.Action(c,actions_df.loc[i])
    print(action)
    

< (Yetam Fen: 5657120867640) -takingAction:69664-> (fabricating) >


## Validate that the action can be completed

In [6]:
action

< (Yetam Fen: 5657120867640) -takingAction:69664-> (fabricating) >

In [7]:
action.job

{'status': 'pending',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'name': 'takingAction',
 'weight': '69664',
 'actionType': 'fabricating',
 'created_at': '69662'}

In [8]:
action.validate_action_time(time)

False

the validation returns true/false and simply confirms that the weight is larger than the current UTC time. 

## Complete the action

In [9]:
action.action.get('augments_self_properties')

In the table below `og` is the origional value, the updated is what the agent's value will change to. 

In [10]:
pd.DataFrame([action.agent, action.resolve_augments_self_properties()], index = ['og','updated'])

KeyError: 'augments_self_properties'

In [10]:
action.add_updates_to_c(time)
action.c.stack

["g.V().has('objid','2909803569658').property('health',0.7).property('wealth',0.522)",
 "g.V().has('objid','2909803569658').outE('takingAction').has('actionType','healthcare_initiatives').has('weight','8014').property('status','resolved')",
 "g.V().has('objid','2909803569658').property('isIdle','true')"]

In [11]:
action.data

{'nodes': [{'objid': '9416220368398',
   'name': 'job',
   'label': 'event',
   'text': 'The pop (Gramerfordmu Rostantrohat) has completed healthcare_initiatives',
   'visibleTo': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
   'time': 12108,
   'username': 'event'}],
 'edges': [{'node1': '2909803569658',
   'node2': '9416220368398',
   'label': 'completed'}]}

Veryify that actions are resolved or not

In [12]:
action.resolve_action()

In [13]:
time.get_global_actions()
actions_df = pd.DataFrame(time.actions)
actions_df 

""


If the `actions_df` is empty then the action has resolved. 

## Actions that involve building